In [1]:
import pandas as pd
import datetime as dt
import numpy as np

In [4]:
#DEFINZIONE VARIABILI E FUZIONI
d_parser = lambda x: dt.datetime.strptime(x, '%Y-%m-%d %H:%M:%S')



#CARICAMENTO DATI
tp = pd.read_csv("C:/Users/11622901/OneDrive - TIM/Team performance/Reports/Lead/WCB/Analisi_Lead_ok.csv",
                       sep=None, engine="python", encoding = "utf16", parse_dates=["Timestamp"], date_parser=d_parser,
                       usecols=["Timestamp", "Campaign", "Media", "Placement", "Banner/Adgroup", "Page", "No Repeats"])

#TRASFORMAZIONE DATI
filt = (tp["No Repeats"] == "Yes") & (tp["Timestamp"].dt.date < dt.date.today())
tp = tp[filt]

tp["Timestamp"] = tp["Timestamp"].dt.date
tp["SubOrder"] = 1
tp.fillna("Missing", inplace=True)

tp_group = tp.groupby(["Timestamp","Page","Campaign","Media","Placement","Banner/Adgroup"])["SubOrder"].sum().reset_index()



mese = tp_group["Timestamp"].max().strftime("%m")
anno = tp_group["Timestamp"].max().strftime("%Y")
giorno = tp_group["Timestamp"].max().strftime("%d")


tp_group["Lead"] = tp_group["Page"][tp_group["Page"].str.contains("OK|KO|DOUBLE|SPAM|CHIAMAMI|CHIAMAMI GRATIS|PROCEDI|FLOAT|VERIFICA E ATTIVA")].str.split("_").str[-1]
tp_group.loc[~tp_group["Lead"].isin(["OK","KO","DOUBLE","SPAM","CHIAMAMI","CHIAMAMI GRATIS","PROCEDI","FLOAT","VERIFICA E ATTIVA"]), "Lead"] = "LANDING"

tp_group["Page"] = tp_group["Page"].str.split('://').str.get(1).str.split('_').str.get(0)

columns_name = ["Timestamp","Page","Campaign","Media","Placement","Banner/Adgroup","Lead","SubOrder","Key"]
tp_group = tp_group.reindex(columns=columns_name)


tp_group.to_excel(f"C:/Users/11622901/OneDrive - TIM/Team performance/Reports/Lead/Lead DEM/Dati/dati_{anno}_{mese}_{giorno}.xlsx", index=False)


### CARTELLE URL
liv = tp_group["Page"].str.split('/', expand=True)
liv["URL"] = tp_group["Page"]
liv["URL"] = liv["URL"].str.split('?').str.get(0)
liv = liv[["URL", 1,2]]
liv.columns = ["URL","L1","L2"]

liv.sort_values(by="URL",inplace=True)
liv.drop_duplicates(subset="URL", keep="first", inplace= True)

liv.to_excel(f"C:/Users/11622901/OneDrive - TIM/Team performance/Reports/Lead/Lead DEM/Livelli/livelli_{anno}_{mese}_{giorno}.xlsx", index=False)


In [254]:
path = pd.read_csv("C:/Users/11622901/JupyterLab/Sources/pathtoconversion.csv",
                       sep=",", engine="python", encoding = "utf-8-sig",usecols=["Journey pattern", "Pattern Count"])

In [255]:
path["Splitted"] = path.loc[:,"Journey pattern"].str.split("->").str[:-1]
path["Last touch"] = path.loc[:,"Splitted"].str[-1]
path["First touch"] = path.loc[:,"Splitted"].str[0]

In [176]:
'''path["Splitted"] = path.loc[:,"Journey pattern"].str.split("->").str[:-1]
path["Last touch"] = path.loc[:,"Journey pattern"].str.split("->").str[-2]
path["First touch"] = path.loc[:,"Journey pattern"].str.split("->").str[1]
path = path.loc[path["Last touch"].str.contains("FANPLAYR", na=False)]'''

In [256]:
path["Path length"] = path["Splitted"].apply(lambda x: len(x))

In [257]:
path["Journey pattern"] = path.loc[:,"Splitted"]

In [258]:
path = path.loc[:,["Journey pattern","Path length","First touch","Last touch","Pattern Count"]]

In [259]:
path["-1"] = path.loc[:,"Journey pattern"].str[-1]
path["-2"] = path.loc[:,"Journey pattern"].str[-2]
path["-3"] = path.loc[:,"Journey pattern"].str[-3]
path["-4"] = path.loc[:,"Journey pattern"].str[-4]
path["-5"] = path.loc[:,"Journey pattern"].str[-5]
path["-6"] = path.loc[:,"Journey pattern"].str[-6]
path["-7"] = path.loc[:,"Journey pattern"].str[-7]
path["-8"] = path.loc[:,"Journey pattern"].str[-8]
path["-9"] = path.loc[:,"Journey pattern"].str[-9]

In [208]:
path.to_excel("C:/Users/11622901/JupyterLab/Sources/path_analysis.xlsx", index=False)

In [287]:
path.groupby(["Path length","First touch"]).agg({"Pattern Count":"sum","Journey pattern":"count"}).rename(columns={"Pattern Count":"Conversioni","Journey pattern":"N. path"}).reset_index().to_excel("C:/Users/11622901/JupyterLab/Sources/path_analysis_first.xlsx", index=False)

In [288]:
path.groupby(["Path length","Last touch"]).agg({"Pattern Count":"sum","Journey pattern":"count"}).rename(columns={"Pattern Count":"Conversioni","Journey pattern":"N. path"}).reset_index().to_excel("C:/Users/11622901/JupyterLab/Sources/path_analysis_last.xlsx", index=False)

In [295]:
path.groupby(["Path length","Last touch"]).agg({"Pattern Count":"max"}).sort_values(by="Pattern Count", ascending=False)

Pattern Count
Path length Last touch                             
1           Unknown                           817.0
            Google AdWords (Click)            564.0
2            Google AdWords (Click)           175.0
1           Empty Path                        161.0
2            FANPLAYR (Click)                 139.0
...                                             ...
6            Facile.it (Impr.)                  1.0
2            Segugio (Click)                    1.0
6            Google AdWords (Impr.)             1.0
             Ketchup (Click)                    1.0
5            Outbrain (Click)                   1.0

[248 rows x 1 columns]